Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 53.8 MB/s eta 0:00:00


Cirq ile örnekler

In [2]:
# cirq_examples.py
# Gereksinimler: pip install cirq numpy
import numpy as np
import cirq

# Pauli matrisleri
I = np.array([[1,0],[0,1]], dtype=complex)
X = np.array([[0,1],[1,0]], dtype=complex)
Y = np.array([[0,-1j],[1j,0]], dtype=complex)
Z = np.array([[1,0],[0,-1]], dtype=complex)
paulis = [I, X, Y, Z]

def pauli_coeffs(rho):
    """Tek qubit için c_i = Tr(rho * sigma_i). Döndürür: [c0,c1,c2,c3]"""
    return [np.trace(rho @ P).real for P in paulis]

def reconstruct_from_paulis(coeffs):
    """rho = 1/2 * sum_i c_i * sigma_i"""
    rho = sum(c * P for c, P in zip(coeffs, paulis)) / 2.0
    return rho

# --------- Örnek 1: tek qubit rho = 3/4 |0><0| + 1/4 |1><1| ----------
rho_single = 0.75 * np.array([[1,0],[0,0]], dtype=complex) + 0.25 * np.array([[0,0],[0,1]], dtype=complex)
print("rho single:\n", rho_single)
c = pauli_coeffs(rho_single)
print("Pauli katsayıları c0..c3:", c)
rho_recon = reconstruct_from_paulis(c)
print("Yeniden oluşturulan rho (Pauli gösterimiyle):\n", np.round(rho_recon, 6))

# --------- Örnek 2: H⊗H |00> durumunun Pauli katsayıları ----------
# Oluştur H|0> ⊗ H|0>
H = 1/np.sqrt(2) * np.array([[1,1],[1,-1]], dtype=complex)
zero = np.array([1,0], dtype=complex)
psi = np.kron(H @ zero, H @ zero)          # state vector
rho2 = np.outer(psi, np.conjugate(psi))    # density matrix 4x4

# Pauli tensör ürünleri c_{ij} = Tr(rho * sigma_i ⊗ sigma_j)
def two_qubit_pauli_coeffs(rho):
    coeffs = np.zeros((4,4), dtype=float)
    for i, P1 in enumerate(paulis):
        for j, P2 in enumerate(paulis):
            coeffs[i,j] = np.trace(rho @ np.kron(P1, P2)).real
    return coeffs

c2 = two_qubit_pauli_coeffs(rho2)
print("\nİki qubit Pauli katsayı matrisi c_{ij}:\n", np.round(c2,4))
# kontrol: |c11|+|c22|+|c33|
sum_c = abs(c2[1,1]) + abs(c2[2,2]) + abs(c2[3,3])
print(" |c11|+|c22|+|c33| =", sum_c, " -> ayrılabilirlik kriteri:", "Ayrılabilir" if sum_c <= 1 else "Dolaşık")

# --------- Örnek 3: Bell state B00 Schmidt ayrıştırma ----------
bell = 1/np.sqrt(2) * (np.kron([1,0],[1,0]) + np.kron([0,1],[0,1]))
rho_bell = np.outer(bell, np.conjugate(bell))

# kısmi iz al ve özdeğerleri hesapla (rho_A)
def partial_trace_A(rho, dimA=2, dimB=2):
    # genel 2x2 sistem için hızlı kısmi iz
    rhoA = np.zeros((dimA,dimA), dtype=complex)
    for i in range(dimB):
        # select block i,i of size dimA
        idx = slice(i*dimA, (i+1)*dimA)
        rhoA += rho[idx, idx]
    return rhoA

rhoA_bell = partial_trace_A(rho_bell)
eigvals = np.linalg.eigvals(rhoA_bell)
print("\nBell state rho_A özdeğerleri (Schmidt kareleri):", np.round(np.real(eigvals),6))
print("Schmidt sayısı (pozitif özdeğerler):", sum(1 for ev in eigvals if np.real(ev) > 1e-9))


rho single:
 [[0.75+0.j 0.  +0.j]
 [0.  +0.j 0.25+0.j]]
Pauli katsayıları c0..c3: [np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(0.5)]
Yeniden oluşturulan rho (Pauli gösterimiyle):
 [[0.75+0.j 0.  +0.j]
 [0.  +0.j 0.25+0.j]]

İki qubit Pauli katsayı matrisi c_{ij}:
 [[1. 1. 0. 0.]
 [1. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
 |c11|+|c22|+|c33| = 0.9999999999999996  -> ayrılabilirlik kriteri: Ayrılabilir

Bell state rho_A özdeğerleri (Schmidt kareleri): [0.5 0.5]
Schmidt sayısı (pozitif özdeğerler): 2


Qiskit ile örnekler

In [3]:
# qiskit_examples.py
# Gereksinimler: pip install qiskit numpy
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, DensityMatrix, partial_trace
# Pauli matrisleri (NumPy)
I = np.array([[1,0],[0,1]], dtype=complex)
X = np.array([[0,1],[1,0]], dtype=complex)
Y = np.array([[0,-1j],[1j,0]], dtype=complex)
Z = np.array([[1,0],[0,-1]], dtype=complex)
paulis = [I,X,Y,Z]

def pauli_coeffs_from_rho(rho):
    return [np.trace(rho @ P).real for P in paulis]

# ---------- Tek qubit örneği ----------
rho_single = 0.75 * np.array([[1,0],[0,0]], dtype=complex) + 0.25 * np.array([[0,0],[0,1]], dtype=complex)
print("rho_single:\n", rho_single)
c = pauli_coeffs_from_rho(rho_single)
print("Pauli c0..c3:", c)
rho_recon = sum(ci * P for ci,P in zip(c, paulis)) / 2.0
print("Reconstructed rho:\n", np.round(rho_recon,6))

# ---------- H⊗H |00> ----------
# Qiskit Statevector ile oluştur
qc = QuantumCircuit(2)
# apply H on both qubits
qc.h(0)
qc.h(1)
sv = Statevector.from_instruction(qc)
rho2 = DensityMatrix(sv).data  # numpy array 4x4
# compute c_{ij}
c2 = np.zeros((4,4))
for i,P1 in enumerate(paulis):
    for j,P2 in enumerate(paulis):
        c2[i,j] = np.trace(rho2 @ np.kron(P1,P2)).real
print("\nTwo-qubit c_{ij} (Qiskit):\n", np.round(c2,4))
sum_c = abs(c2[1,1]) + abs(c2[2,2]) + abs(c2[3,3])
print(" |c11|+|c22|+|c33| =", sum_c, " ->", "Ayrılabilir" if sum_c <= 1 else "Dolaşık")

# ---------- Bell B00 Schmidt ----------
# Build Bell state via circuit
qc2 = QuantumCircuit(2)
qc2.h(0)
qc2.cx(0,1)
sv_bell = Statevector.from_instruction(qc2)
rho_bell = DensityMatrix(sv_bell).data
# partial trace (qiskit function)
rhoA = partial_trace(DensityMatrix(rho_bell), [1]).data   # trace out qubit 1 (B), left A
eigvals = np.linalg.eigvals(rhoA)
print("\nBell rho_A özdeğerleri (Qiskit):", np.round(np.real(eigvals),6))
print("Schmidt sayısı:", sum(1 for ev in eigvals if np.real(ev) > 1e-9))


rho_single:
 [[0.75+0.j 0.  +0.j]
 [0.  +0.j 0.25+0.j]]
Pauli c0..c3: [np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(0.5)]
Reconstructed rho:
 [[0.75+0.j 0.  +0.j]
 [0.  +0.j 0.25+0.j]]

Two-qubit c_{ij} (Qiskit):
 [[1. 1. 0. 0.]
 [1. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
 |c11|+|c22|+|c33| = 0.9999999999999996  -> Ayrılabilir

Bell rho_A özdeğerleri (Qiskit): [0.5 0.5]
Schmidt sayısı: 2


QuTiP ile örnekler

In [4]:
# qutip_examples.py
# Gereksinimler: pip install qutip numpy
import numpy as np
import qutip as qt

# Pauli operatörleri QuTiP ile
I = qt.qeye(2)
X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()
paulis_q = [I, X, Y, Z]

def pauli_coeffs_qutip(rho_qobj):
    coeffs = []
    for P in paulis_q:
        val = (rho_qobj * P).tr()
        coeffs.append(np.real(val))
    return coeffs

# ---------- Tek qubit rho ----------
rho_single_q = 0.75 * qt.ket2dm(qt.basis(2,0)) + 0.25 * qt.ket2dm(qt.basis(2,1))
print("rho_single (QuTiP):\n", rho_single_q)
c = pauli_coeffs_qutip(rho_single_q)
print("Pauli c0..c3 (QuTiP):", c)
# reconstruct
rho_recon_q = sum(ci * P for ci,P in zip(c, paulis_q)) * 0.5
print("Reconstructed rho (QuTiP):\n", rho_recon_q)

# ---------- H⊗H |00> ----------
H = (1/np.sqrt(2)) * qt.Qobj(np.array([[1,1],[1,-1]], dtype=complex))
zero = qt.basis(2,0)
psi = qt.tensor(H * zero, H * zero)  # state vector
rho2 = psi.proj()
# compute c_{ij}
c2 = np.zeros((4,4))
for i,P1 in enumerate(paulis_q):
    for j,P2 in enumerate(paulis_q):
        M = qt.tensor(P1, P2)
        c2[i,j] = np.real((rho2 * M).tr())
print("\nTwo-qubit c_{ij} (QuTiP):\n", np.round(c2,4))
sum_c = abs(c2[1,1]) + abs(c2[2,2]) + abs(c2[3,3])
print(" |c11|+|c22|+|c33| =", sum_c, " ->", "Ayrılabilir" if sum_c <= 1 else "Dolaşık")

# ---------- Bell B00 Schmidt (QuTiP partial trace) ----------
bell = (qt.basis(2,0) * qt.basis(2,0).dag() + qt.basis(2,1) * qt.basis(2,1).dag()) # yanlış kullanım; yapalım düzgün
# do Bell properly:
bell_state = (qt.tensor(qt.basis(2,0), qt.basis(2,0)) + qt.tensor(qt.basis(2,1), qt.basis(2,1))).unit()
rho_bell = bell_state.proj()
rhoA = rho_bell.ptrace(0)   # partial trace over subsystem 1 (keep subsystem 0)
eigvals = np.real(rhoA.eigenenergies())
print("\nBell rho_A özdeğerleri (QuTiP):", np.round(eigvals,6))
print("Schmidt sayısı:", sum(1 for ev in eigvals if ev > 1e-9))


rho_single (QuTiP):
 Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0.75 0.  ]
 [0.   0.25]]
Pauli c0..c3 (QuTiP): [1.0, 0.0, 0.0, 0.5]
Reconstructed rho (QuTiP):
 Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0.75 0.  ]
 [0.   0.25]]

Two-qubit c_{ij} (QuTiP):
 [[1. 1. 0. 0.]
 [1. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
 |c11|+|c22|+|c33| = 0.9999999999999996  -> Ayrılabilir

Bell rho_A özdeğerleri (QuTiP): [0.5 0.5]
Schmidt sayısı: 2
